Comprobación de PyTorch y uso de la GPU

In [ ]:
import torch

print(f"Versión de PyTorch: {torch.__version__}")
print(f"Versión de CUDA en PyTorch: {torch.version.cuda}")
print(f"¿CUDA está disponible?: {torch.cuda.is_available()}")
print(torch.cuda.get_device_name(0))

Entrenamiento de las redes

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")

results = model.train(data="./datasets/bod_v1/data.yaml", 
                      device=0,
                      batch=8, 
                      epochs=100,
                      imgsz=1024, 
                      optimizer='SGD', 
                      lr0=0.01, 
                      lrf=0.1, 
                      weight_decay=0.0005, 
                      task="detect")

Librerías, paquetes y funciones importadas

In [ ]:
def getColor(class_name):
    switch = {
        'player': (255, 0, 0),       # Azul
        'basketball': (0, 165, 255), # Naranja
        'rim': (0, 0, 255),          # Rojo
        'made-shot': (0, 255, 0)     # Verde
    }
    return switch.get(class_name, (0, 0, 0)) 

def drawBBox(frame, x1, y1, x2, y2, label, class_name):
    color = getColor(class_name)
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
    cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

def drawPosition(frame, position, position_label):
    cv2.ellipse(frame, (int(position[0]), int(position[1])), (9, 3), 0, 0, 360, (0, 0, 255), -1)
    cv2.putText(frame, position_label, (int(position[0]) - 50, int(position[1]) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

In [ ]:
import cv2
from ultralytics import YOLO

object_detector = YOLO("./runs/detect/bod_v1/weights/best.pt")

video_path = "../clips/ClipLF1.mp4"
output_path = "../output/ClipLF1_output.mp4"

cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*"avc1")
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    object_detection_results = object_detector(frame)

    # Análisis de las detecciones de jugadores
    for result in object_detection_results[0].boxes.data.tolist():  # Obtener los resultados como lista
        x1, y1, x2, y2, conf, cls = result  # Coordenadas, confianza y clase
        cls = int(cls)

        # Filtrar solo por las clases deseadas
        if object_detector.names[cls] in ['player', 'basketball', 'rim', 'made-shot'] and conf > 0.5:

            # Calcular la posición como el punto medio del borde inferior de la bbox
            position = (int((x1 + x2) / 2), y2)
            position_label = f"x:{int(position[0])} y:{int(position[1])}"

            label = f"{object_detector.names[cls]} {conf:.2f}"

            drawBBox(frame, x1, y1, x2, y2, label, object_detector.names[cls])

            # Dibujar la posición en el frame
            if object_detector.names[cls] == 'player' or object_detector.names[cls] == 'basketball':
                drawPosition(frame, position, position_label)


    # Mostrar el frame procesado en pantalla
    cv2.imshow('Resultados', frame)

    # Esperar por una tecla: espacio para avanzar, 'q' para salir
    key = cv2.waitKey(0) & 0xFF  # Espera indefinidamente hasta que se presione una tecla
    if key == ord('q'):
        break
    elif key == ord(' '):  # Espacio para continuar
        # Escribir el frame procesado en el video de salida
        out.write(frame)


# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado en {output_path}")
